# Data exploration and analysis: GooglePlayStoreApps

With a dataset with information from the applications in the Google Play Store, the aim is to obtain a model capable of predicting the rating for applications not yet rated by users.
Since the rating can be a real number between 1 and 5, this will be considered a regression problem.

There are two datasets:
- one with the data of the apps in the playstore, including information about the applications (such as category, rating and number of reviews, among others);
- another with the reviews made by users, including the text and the value of the sentiment/polarity identified in those comments.

To create the model, a series of tasks will first be carried out:
- explore the data and relationships between different features, identifying missing values and outliers;
- creation of a pipeline with the tasks to perform prior to model training;
- creation of various models, training and testing;
- use of metrics to select the final model.

# Tabla de contenidos

1. [Exploratory data analysis](#eda)
    1. [Application data](#dataapps)
    2. [Eliminate duplicate rows](#removeduplicates)
    3. [Dealing with missing values](#missingvalues)
    4. [Analysis of numerical features](#numericfeatures)

In [2]:
# Imports

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# Reading the files.

# -- Data about apps.
data_apps = pd.read_csv("data/googleplaystore.csv")

# -- Reviews made by users and sentiment analysis values.
data_reviews = pd.read_csv("data/googleplaystore_user_reviews.csv")

#### Regarding the reviews dataset

We are going to concentrate mainly on the data of the apps (data_apps) to try to predict the rating. While the data about reviews (the polarity of mixed sentiments in those reviews) might help us predict a Rating, for now we're going to use only app values to do so.

Especially since we only have data from 1074 apps, compared to the 8182 apps in the final dataset (removed duplicate rows and also apps without Rating) => reviews for less than 15% of the apps.

In [5]:
## Information about de reviews
data_reviews

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
...,...,...,...,...,...
64290,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN
64291,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN
64292,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN
64293,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN


In [6]:
data_reviews.groupby("App").count()

,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
App,,,,
10 Best Foods for You,194,194,194,194
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,40,40,40,40
11st,39,40,40,40
1800 Contacts - Lens Store,80,80,80,80
1LINE – One Line with One Touch,38,38,38,38
...,...,...,...,...
Hotspot Shield Free VPN Proxy & Wi-Fi Security,34,34,34,34
Hotstar,32,32,32,32
Hotwire Hotel & Car Rental App,33,33,33,33


# Exploratory data analysis <a name="eda"></a>

Before creating the model, we will analyze the data. This involves looking at the values they can take, recategorizing the data if necessary, and looking at missing values and outliers and dealing with them in some way.

## Application data <a name="dataapps"></a>

In this dataset we have the following features:
- App: indicates the name of the application.
- Category: the category of the app (Art and design, Family, Sports, etc.).
- Rating (variable to predict): indicates the rating of the application, a real value (with a decimal) that can range from 1 to 5.
- Reviews: the number of reviews that the app has.
- Size: the size of the application.
- Installs: the number of installations of the apps.
- Type: if it is free or paid.
- Price: the price.
- Content Rating: the content rating (suitable for everyone, over 18, etc).
- Genres: the genres of the application.
- Last Updated: when it was last updated.
- Current Ver: the version of the application.
- Android Ver: the version of android in which it can be used.

In [7]:
data_apps

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,07/01/2018,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,15/01/2018,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,01/08/2018,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,08/06/2018,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,20/06/2018,1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,06/07/2018,1,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,20/01/2017,1,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,19/01/2015,Varies with device,Varies with device
10840,iHoroscope - 2018 Daily Horoscope & Astrology,LIFESTYLE,4.5,398307,19M,"10,000,000+",Free,0,Everyone,Lifestyle,25/07/2018,Varies with device,Varies with device


Let's see the information about the type of the features.
We have 10842 rows. Most features are type Object. Rating is a float, and Reviews is an integer. Other features that could be numeric are:
- Size
- Installs
- Price

In [8]:
data_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10842 entries, 0 to 10841
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10842 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9368 non-null   float64
 3   Reviews         10842 non-null  int64  
 4   Size            10842 non-null  object 
 5   Installs        10842 non-null  object 
 6   Type            10841 non-null  object 
 7   Price           10842 non-null  object 
 8   Content Rating  10842 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10842 non-null  object 
 11  Current Ver     10834 non-null  object 
 12  Android Ver     10840 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.1+ MB


We can also see that we have some missing values in some columns. We will deal with these missing values later.

## Removing duplicate rows <a name="removeduplicates"></a>

Before we start dealing with missing values and looking at features, let's make sure we don't have duplicate rows for apps.

Number of duplicate rows: 1181

In [9]:
# Duplicated rows for app name => 1181
data_apps[data_apps.duplicated(['App'])]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
229,Quick PDF Scanner + OCR FREE,BUSINESS,4.2,80805,Varies with device,"5,000,000+",Free,0,Everyone,Business,26/02/2018,Varies with device,4.0.3 and up
236,Box,BUSINESS,4.2,159872,Varies with device,"10,000,000+",Free,0,Everyone,Business,31/07/2018,Varies with device,Varies with device
239,Google My Business,BUSINESS,4.4,70991,Varies with device,"5,000,000+",Free,0,Everyone,Business,24/07/2018,2.19.0.204537701,4.4 and up
256,ZOOM Cloud Meetings,BUSINESS,4.4,31614,37M,"10,000,000+",Free,0,Everyone,Business,20/07/2018,4.1.28165.0716,4.0 and up
261,join.me - Simple Meetings,BUSINESS,4.0,6989,Varies with device,"1,000,000+",Free,0,Everyone,Business,16/07/2018,4.3.0.508,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10715,FarmersOnly Dating,DATING,3.0,1145,1.4M,"100,000+",Free,0,Mature 17+,Dating,25/02/2016,2.2,4.0 and up
10720,Firefox Focus: The privacy browser,COMMUNICATION,4.4,36981,4.0M,"1,000,000+",Free,0,Everyone,Communication,06/07/2018,5.2,5.0 and up
10730,FP Notebook,MEDICAL,4.5,410,60M,"50,000+",Free,0,Everyone,Medical,24/03/2018,2.1.0.372,4.4 and up
10753,Slickdeals: Coupons & Shopping,SHOPPING,4.5,33599,12M,"1,000,000+",Free,0,Everyone,Shopping,30/07/2018,3.9,4.4 and up


Example with ROBLOX.
We can see that the number of reviews is the value that is changing => regarding that, we can keep the highest.
It also changes the category (!). Let's see if this happens for many apps.

In [11]:
data_apps[data_apps["App"] == "ROBLOX"].sort_values(by="Reviews", ascending=False)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2206,ROBLOX,FAMILY,4.5,4450890,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,31/07/2018,2.347.225742,4.1 and up
2088,ROBLOX,FAMILY,4.5,4450855,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,31/07/2018,2.347.225742,4.1 and up
1870,ROBLOX,GAME,4.5,4449910,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,31/07/2018,2.347.225742,4.1 and up
2016,ROBLOX,FAMILY,4.5,4449910,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,31/07/2018,2.347.225742,4.1 and up
1841,ROBLOX,GAME,4.5,4449882,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,31/07/2018,2.347.225742,4.1 and up
1748,ROBLOX,GAME,4.5,4448791,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,31/07/2018,2.347.225742,4.1 and up
1653,ROBLOX,GAME,4.5,4447388,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,31/07/2018,2.347.225742,4.1 and up
1701,ROBLOX,GAME,4.5,4447346,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,31/07/2018,2.347.225742,4.1 and up
4527,ROBLOX,FAMILY,4.5,4443407,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,31/07/2018,2.347.225742,4.1 and up


How many apps have multiple categories? => 85.

Since there are just a few, in this case we are going to choose to keep the last category that was assigned to the app.
We are left with the most up-to-date entry regarding "Reviews".

In [12]:
temp = data_apps.groupby(["App"])["Category"].unique()
temp[temp.str.len() > 1]

App
8 Ball Pool                                                  [GAME, SPORTS]
A&E - Watch Full Episodes of TV Shows               [ENTERTAINMENT, FAMILY]
Angry Birds 2                                                [GAME, FAMILY]
Babbel – Learn Languages                                [EDUCATION, FAMILY]
Barbie™ Fashion Closet                                       [GAME, FAMILY]
                                                             ...           
Video Editor                                        [FAMILY, VIDEO_PLAYERS]
WWE                                                 [ENTERTAINMENT, FAMILY]
YouTube Gaming                                      [ENTERTAINMENT, FAMILY]
YouTube Kids                                        [ENTERTAINMENT, FAMILY]
busuu: Learn Languages - Spanish, English & More        [EDUCATION, FAMILY]
Name: Category, Length: 85, dtype: object

In [13]:
data_apps = data_apps.sort_values('Reviews', ascending=False).drop_duplicates('App').sort_index()

# Example
data_apps[data_apps["App"] == "ROBLOX"]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2206,ROBLOX,FAMILY,4.5,4450890,67M,"100,000,000+",Free,0,Everyone 10+,Adventure;Action & Adventure,31/07/2018,2.347.225742,4.1 and up


### Dealing with missing values <a name="missingvalues"></a>
 
We are now going to deal with the missing values in the features. We first calculate the sum of the missing values in each feature, ordered in descending order.

In [14]:
data_apps.isna().sum().sort_values(ascending=False)

Rating            1463
Current Ver          8
Android Ver          2
Category             1
Type                 1
Genres               1
App                  0
Reviews              0
Size                 0
Installs             0
Price                0
Content Rating       0
Last Updated         0
dtype: int64

We have quite a few missing values in Rating, which is the variable to predict, and a few others in Current Ver, Android Ver, Category, Type and Genres. Let's observe and try to solve these missing values.

Let's start with those that only have one missing value: Category, Type, and Genres.

In [15]:
# Let's see the row with missing value in Type
data_apps[data_apps["Type"].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
9148,Command & Conquer: Rivals,FAMILY,NaN,0,Varies with device,0,NaN,0,Everyone 10+,Strategy,28/06/2018,Varies with device,Varies with device


In [16]:
# What kind of values can Type have? ==> "Free" or "Paid".
data_apps["Type"].unique()

array(['Free', 'Paid', nan], dtype=object)

Since Type can only take the values "Free" or "Paid", and the price of that app is 0, we can fill in the missing value as "Free".

In [17]:
# Updating the value of the row 9148, column 6 (Type) por Free
data_apps.iloc[[9148],[6]] = "Free"

# data_apps[data_apps["Type"].isna()] # To check that we don't have that values

Now let's look at the row that has a missing value in Category.
Looking at the row, we can also see that it has a missing value for Genres.

A possible solution could be to delete this row. But first let's see what kind of categories and genres exist, to see if there is any category "Others" or something like that.

In [18]:
data_apps[data_apps["Category"].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,NaN,1.9,19,3.0M,"1,000+",Free,0,Everyone,NaN,11/02/2018,1.0.19,4.0 and up


In [19]:
# Possible values for Category

data_apps["Category"].unique()

array(['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY',
       'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION',
       'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE',
       'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME',
       'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL',
       'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL',
       'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER',
       'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION', nan],
      dtype=object)

In [20]:
data_apps["Category"].value_counts()

FAMILY                 1876
GAME                    945
TOOLS                   829
BUSINESS                420
MEDICAL                 395
PERSONALIZATION         376
PRODUCTIVITY            374
LIFESTYLE               369
FINANCE                 345
SPORTS                  326
COMMUNICATION           315
HEALTH_AND_FITNESS      288
PHOTOGRAPHY             281
NEWS_AND_MAGAZINES      254
SOCIAL                  239
BOOKS_AND_REFERENCE     222
TRAVEL_AND_LOCAL        219
SHOPPING                202
DATING                  170
VIDEO_PLAYERS           164
MAPS_AND_NAVIGATION     131
FOOD_AND_DRINK          112
EDUCATION               106
ENTERTAINMENT            87
AUTO_AND_VEHICLES        85
LIBRARIES_AND_DEMO       84
WEATHER                  79
HOUSE_AND_HOME           73
EVENTS                   64
ART_AND_DESIGN           61
PARENTING                60
COMICS                   56
BEAUTY                   53
Name: Category, dtype: int64

Let's look at the possible values for Genres. These seem to be overlapping quite a bit with the Category. You can see that all the values for Category appear as Genres. There are also new values for Genres that seem to indicate subcategories (for example, different types of games).

In [21]:
data_apps["Genres"].unique()

array(['Art & Design', 'Art & Design;Creativity', 'Auto & Vehicles',
       'Beauty', 'Books & Reference', 'Business', 'Comics',
       'Comics;Creativity', 'Communication', 'Dating', 'Education',
       'Education;Creativity', 'Education;Education',
       'Education;Pretend Play', 'Education;Brain Games', 'Entertainment',
       'Entertainment;Brain Games', 'Entertainment;Creativity',
       'Entertainment;Music & Video', 'Events', 'Finance', 'Food & Drink',
       'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Lifestyle;Pretend Play', 'Card', 'Arcade', 'Puzzle',
       'Racing', 'Sports', 'Casual', 'Simulation', 'Adventure', 'Trivia',
       'Action', 'Word', 'Role Playing', 'Strategy', 'Board',
       'Simulation;Education', 'Action;Action & Adventure', 'Music',
       'Casual;Brain Games', 'Educational;Creativity',
       'Puzzle;Brain Games', 'Educational;Education',
       'Casual;Pretend Play', 'Educational;Brain Games',
       'Art & Design;Preten

There doesn't seem to be a category like "Other". Let's delete this row.

In [22]:
# Removing the row with missing value in Category and Genres
data_apps = data_apps.drop([10472])

Let's see the rows with missing values in Android View, and the possible values that feature can take.

In [23]:
data_apps[data_apps["Android Ver"].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
4453,[substratum] Vacuum: P,PERSONALIZATION,4.4,230,11M,"1,000+",Paid,$1.49,Everyone,Personalization,20/07/2018,4.4,NaN
4490,Pi Dark [substratum],PERSONALIZATION,4.5,189,2.1M,"10,000+",Free,0,Everyone,Personalization,27/03/2018,1.1,NaN


In [24]:
data_apps["Android Ver"].unique()

array(['4.0.3 and up', '4.2 and up', '4.4 and up', '2.3 and up',
       '3.0 and up', '4.1 and up', '4.0 and up', '2.3.3 and up',
       'Varies with device', '2.2 and up', '5.0 and up', '6.0 and up',
       '1.6 and up', '1.5 and up', '2.1 and up', '7.0 and up',
       '5.1 and up', '4.3 and up', '4.0.3 - 7.1.1', '2.0 and up',
       '3.2 and up', '4.4W and up', '7.1 and up', '7.0 - 7.1.1',
       '8.0 and up', '5.0 - 8.0', '3.1 and up', '2.0.1 and up',
       '4.1 - 7.1.1', nan, '5.0 - 6.0', '1.0 and up', '2.2 - 7.1.1',
       '5.0 - 7.1.1'], dtype=object)

For simplicity, let's fill in the missing values with the value "Varies with device".

In [25]:
data_apps["Android Ver"] = data_apps["Android Ver"].fillna(value="Varies with device")

Let's look at the rows with missing values in Current Ver, and also the possible values for Current Ver. There is also a "Varies with device" value here. For simplicity, we also fill in these missing values with "Varies with device".

In [26]:
data_apps[data_apps["Current Ver"].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
15,Learn To Draw Kawaii Characters,ART_AND_DESIGN,3.2,55,2.7M,"5,000+",Free,0,Everyone,Art & Design,06/06/2018,NaN,4.2 and up
1553,Market Update Helper,LIBRARIES_AND_DEMO,4.1,20145,11k,"1,000,000+",Free,0,Everyone,Libraries & Demo,12/02/2013,NaN,1.5 and up
6322,Virtual DJ Sound Mixer,TOOLS,4.2,4010,8.7M,"500,000+",Free,0,Everyone,Tools,10/05/2017,NaN,4.0 and up
6803,BT Master,FAMILY,NaN,0,222k,100+,Free,0,Everyone,Education,06/11/2016,NaN,1.6 and up
7333,Dots puzzle,FAMILY,4.0,179,14M,"50,000+",Paid,$0.99,Everyone,Puzzle,18/04/2018,NaN,4.0 and up
7407,Calculate My IQ,FAMILY,NaN,44,7.2M,"10,000+",Free,0,Everyone,Entertainment,03/04/2017,NaN,2.3 and up
7730,UFO-CQ,TOOLS,NaN,1,237k,10+,Paid,$0.99,Everyone,Tools,04/07/2016,NaN,2.0 and up
10342,La Fe de Jesus,BOOKS_AND_REFERENCE,NaN,8,658k,"1,000+",Free,0,Everyone,Books & Reference,31/01/2017,NaN,3.0 and up


In [27]:
data_apps["Current Ver"].value_counts()

Varies with device    1055
1                      830
1.1                    272
1.2                    183
2                      163
                      ... 
0.1.187945513            1
2.2.194                  1
68.0.3440.91             1
4.8.2.2195               1
2.0.148.0                1
Name: Current Ver, Length: 2771, dtype: int64

In [28]:
data_apps["Current Ver"] = data_apps["Current Ver"].fillna(value="Varies with device")

We still need to see what happens to the missing values for our variable to predict, "Rating". Let's see how many Reviews are associated with it.
- Rows that lack rating: 1474
- Rows that lack rating and have some review: 878
- Rows that lack rating and have no reviews: 596

Not all of them have reviews to be able to use those values of the reviews (from the other dataset) in order to predict some Rating.

In [29]:
# rows a las que les falta rating: 1474 
# data_apps[data_apps["Rating"].isna()] 

# rows a las que les falta rating y tienen algún review: 878 
# data_apps[(data_apps["Rating"].isna()) & (data_apps["Reviews"] > 0)] 

# rows a las que les falta rating y no tienen reviews: 596  
# data_apps[(data_apps["Rating"].isna()) & (data_apps["Reviews"] == 0)] 

In this case we have two options:
- delete rows;
- use some Imputer to complete the data (KNNImputer, for example) => taking into account what this means (completing the missing data using the nearest neighbors).

Let's try both approaches and compare the results.